# 自定义中间件
通过实现能在智能体执行流程特定节点运行的钩子函数，即可构建自定义中间件。

## 钩子函数
中间件提供两种类型的钩子函数，用于拦截智能体的执行过程：

<CardGroup cols={2}>
  <Card title="节点式钩子函数" icon="share-nodes" href="#node-style-hooks">
    在特定执行节点按顺序运行。
  </Card>

  <Card title="包装式钩子函数" icon="container-storage" href="#wrap-style-hooks">
    围绕每次模型调用或工具调用运行。
  </Card>
</CardGroup>

### 节点式钩子函数
在特定执行节点按顺序运行，适用于日志记录、参数校验和状态更新场景。

**可用钩子函数：**
* `before_agent` - 智能体启动前执行（每次调用仅执行一次）
* `before_model` - 每次模型调用前执行
* `after_model` - 每次模型返回响应后执行
* `after_agent` - 智能体执行完成后执行（每次调用仅执行一次）

**代码示例：**

<Tabs>
  <Tab title="装饰器写法">
    ```python  theme={null}
    from langchain.agents.middleware import before_model, after_model, AgentState
    from langchain.messages import AIMessage
    from langgraph.runtime import Runtime
    from typing import Any


    @before_model(can_jump_to=["end"])
    def check_message_limit(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
        if len(state["messages"]) >= 50:
            return {
                "messages": [AIMessage("对话已达到消息数量上限。")],
                "jump_to": "end"
            }
        return None

    @after_model
    def log_response(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
        print(f"模型返回结果: {state['messages'][-1].content}")
        return None
    ```
  </Tab>

  <Tab title="类写法">
    ```python  theme={null}
    from langchain.agents.middleware import AgentMiddleware, AgentState, hook_config
    from langchain.messages import AIMessage
    from langgraph.runtime import Runtime
    from typing import Any

    class MessageLimitMiddleware(AgentMiddleware):
        def __init__(self, max_messages: int = 50):
            super().__init__()
            self.max_messages = max_messages

        @hook_config(can_jump_to=["end"])
        def before_model(self, state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
            if len(state["messages"]) == self.max_messages:
                return {
                    "messages": [AIMessage("对话已达到消息数量上限。")],
                    "jump_to": "end"
                }
            return None

        def after_model(self, state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
            print(f"模型返回结果: {state['messages'][-1].content}")
            return None
    ```
  </Tab>
</Tabs>

### 包装式钩子函数
拦截执行流程并控制处理器的调用时机，适用于重试、缓存和数据转换场景。

你可以决定处理器的调用次数：零次（短路执行）、一次（正常流程）或多次（重试逻辑）。

**可用钩子函数：**
* `wrap_model_call` - 围绕每次模型调用执行
* `wrap_tool_call` - 围绕每次工具调用执行

**代码示例：**

<Tabs>
  <Tab title="装饰器写法">
    ```python  theme={null}
    from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
    from typing import Callable


    @wrap_model_call
    def retry_model(
        request: ModelRequest,
        handler: Callable[[ModelRequest], ModelResponse],
    ) -> ModelResponse:
        for attempt in range(3):
            try:
                return handler(request)
            except Exception as e:
                if attempt == 2:
                    raise
                print(f"调用失败，正在进行第 {attempt + 1}/3 次重试，错误信息: {e}")
    ```
  </Tab>

  <Tab title="类写法">
    ```python  theme={null}
    from langchain.agents.middleware import AgentMiddleware, ModelRequest, ModelResponse
    from typing import Callable

    class RetryMiddleware(AgentMiddleware):
        def __init__(self, max_retries: int = 3):
            super().__init__()
            self.max_retries = max_retries

        def wrap_model_call(
            self,
            request: ModelRequest,
            handler: Callable[[ModelRequest], ModelResponse],
        ) -> ModelResponse:
            for attempt in range(self.max_retries):
                try:
                    return handler(request)
                except Exception as e:
                    if attempt == self.max_retries - 1:
                        raise
                    print(f"调用失败，正在进行第 {attempt + 1}/{self.max_retries} 次重试，错误信息: {e}")
    ```
  </Tab>
</Tabs>

## 创建中间件
创建中间件有两种方式：

<CardGroup cols={2}>
  <Card title="基于装饰器的中间件" icon="at" href="#decorator-based-middleware">
    适用于单一钩子函数的简单场景，通过装饰器包装独立函数实现。
  </Card>

  <Card title="基于类的中间件" icon="brackets-curly" href="#class-based-middleware">
    适用于包含多个钩子函数或需要复杂配置的场景，功能更强大。
  </Card>
</CardGroup>

### 基于装饰器的中间件
适用于单一钩子函数的简单场景，通过装饰器包装独立函数即可实现。

**可用装饰器：**
**节点式：**
* [`@before_agent`](https://reference.langchain.com/python/langchain/middleware/#langchain.agents.middleware.before_agent) - 智能体启动前执行（每次调用仅执行一次）
* [`@before_model`](https://reference.langchain.com/python/langchain/middleware/#langchain.agents.middleware.before_model) - 每次模型调用前执行
* [`@after_model`](https://reference.langchain.com/python/langchain/middleware/#langchain.agents.middleware.after_model) - 每次模型返回响应后执行
* [`@after_agent`](https://reference.langchain.com/python/langchain/middleware/#langchain.agents.middleware.after_agent) - 智能体执行完成后执行（每次调用仅执行一次）

**包装式：**
* [`@wrap_model_call`](https://reference.langchain.com/python/langchain/middleware/#langchain.agents.middleware.wrap_model_call) - 用自定义逻辑包装每次模型调用
* [`@wrap_tool_call`](https://reference.langchain.com/python/langchain/middleware/#langchain.agents.middleware.wrap_tool_call) - 用自定义逻辑包装每次工具调用

**便捷装饰器：**
* [`@dynamic_prompt`](https://reference.langchain.com/python/langchain/middleware/#langchain.agents.middleware.dynamic_prompt) - 生成动态系统提示词

**代码示例：**
```python  theme={null}
from langchain.agents.middleware import (
    before_model,
    wrap_model_call,
    AgentState,
    ModelRequest,
    ModelResponse,
)
from langchain.agents import create_agent
from langgraph.runtime import Runtime
from typing import Any, Callable


@before_model
def log_before_model(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    print(f"即将调用模型，当前消息数量: {len(state['messages'])}")
    return None

@wrap_model_call
def retry_model(
    request: ModelRequest,
    handler: Callable[[ModelRequest], ModelResponse],
) -> ModelResponse:
    for attempt in range(3):
        try:
            return handler(request)
        except Exception as e:
            if attempt == 2:
                raise
            print(f"调用失败，正在进行第 {attempt + 1}/3 次重试，错误信息: {e}")

agent = create_agent(
    model="gpt-4o",
    middleware=[log_before_model, retry_model],
    tools=[...],
)
```

**适用场景：**
* 仅需单个钩子函数
* 无需复杂配置
* 快速原型开发

### 基于类的中间件
适用于包含多个钩子函数或需要复杂配置的场景，功能更强大。当你需要为同一个钩子函数同时定义同步和异步实现，或在单个中间件中组合多个钩子函数时，推荐使用类写法。

**代码示例：**
```python  theme={null}
from langchain.agents.middleware import (
    AgentMiddleware,
    AgentState,
    ModelRequest,
    ModelResponse,
)
from langgraph.runtime import Runtime
from typing import Any, Callable

class LoggingMiddleware(AgentMiddleware):
    def before_model(self, state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
        print(f"即将调用模型，当前消息数量: {len(state['messages'])}")
        return None

    def after_model(self, state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
        print(f"模型返回结果: {state['messages'][-1].content}")
        return None

agent = create_agent(
    model="gpt-4o",
    middleware=[LoggingMiddleware()],
    tools=[...],
)
```

**适用场景：**
* 需要为同一个钩子函数定义同步和异步两种实现
* 单个中间件需包含多个钩子函数
* 需要复杂配置（如可配置阈值、自定义模型）
* 需要在多个项目中复用，且需在初始化时配置参数

## 自定义状态模式
中间件可以通过自定义属性扩展智能体的状态，支持以下功能：
* **跨执行流程跟踪状态**：维护计数器、标记或其他在智能体整个生命周期中持续存在的值
* **在钩子函数间共享数据**：将数据从`before_model`钩子函数传递到`after_model`钩子函数，或在不同中间件实例间共享数据
* **实现横切关注点功能**：无需修改智能体核心逻辑，即可添加限流、使用量统计、用户上下文管理或审计日志等功能
* **执行条件决策**：利用累计的状态信息，决定是否继续执行、跳转到不同节点或动态修改执行行为

<Tabs>
  <Tab title="装饰器写法">
    ```python  theme={null}
    from langchain.agents import create_agent
    from langchain.messages import HumanMessage
    from langchain.agents.middleware import AgentState, before_model, after_model
    from typing_extensions import NotRequired
    from typing import Any
    from langgraph.runtime import Runtime


    class CustomState(AgentState):
        model_call_count: NotRequired[int]
        user_id: NotRequired[str]


    @before_model(state_schema=CustomState, can_jump_to=["end"])
    def check_call_limit(state: CustomState, runtime: Runtime) -> dict[str, Any] | None:
        count = state.get("model_call_count", 0)
        if count > 10:
            return {"jump_to": "end"}
        return None


    @after_model(state_schema=CustomState)
    def increment_counter(state: CustomState, runtime: Runtime) -> dict[str, Any] | None:
        return {"model_call_count": state.get("model_call_count", 0) + 1}


    agent = create_agent(
        model="gpt-4o",
        middleware=[check_call_limit, increment_counter],
        tools=[],
    )

    # 传入自定义状态调用智能体
    result = agent.invoke({
        "messages": [HumanMessage("你好")],
        "model_call_count": 0,
        "user_id": "user-123",
    })
    ```
  </Tab>

  <Tab title="类写法">
    ```python  theme={null}
    from langchain.agents import create_agent
    from langchain.messages import HumanMessage
    from langchain.agents.middleware import AgentState, AgentMiddleware
    from typing_extensions import NotRequired
    from typing import Any


    class CustomState(AgentState):
        model_call_count: NotRequired[int]
        user_id: NotRequired[str]


    class CallCounterMiddleware(AgentMiddleware[CustomState]):
        state_schema = CustomState

        def before_model(self, state: CustomState, runtime) -> dict[str, Any] | None:
            count = state.get("model_call_count", 0)
            if count > 10:
                return {"jump_to": "end"}
            return None

        def after_model(self, state: CustomState, runtime) -> dict[str, Any] | None:
            return {"model_call_count": state.get("model_call_count", 0) + 1}


    agent = create_agent(
        model="gpt-4o",
        middleware=[CallCounterMiddleware()],
        tools=[],
    )

    # 传入自定义状态调用智能体
    result = agent.invoke({
        "messages": [HumanMessage("你好")],
        "model_call_count": 0,
        "user_id": "user-123",
    })
    ```
  </Tab>
</Tabs>

## 执行顺序
当使用多个中间件时，需了解它们的执行顺序：
```python  theme={null}
agent = create_agent(
    model="gpt-4o",
    middleware=[middleware1, middleware2, middleware3],
    tools=[...],
)
```

<Accordion title="执行流程">
  **前置钩子函数按顺序执行：**
  1. `middleware1.before_agent()`
  2. `middleware2.before_agent()`
  3. `middleware3.before_agent()`

  **智能体循环启动**
  4. `middleware1.before_model()`
  5. `middleware2.before_model()`
  6. `middleware3.before_model()`

  **包装式钩子函数按函数调用方式嵌套执行：**
  7. `middleware1.wrap_model_call()` → `middleware2.wrap_model_call()` → `middleware3.wrap_model_call()` → 模型调用

  **后置钩子函数按逆序执行：**
  8. `middleware3.after_model()`
  9. `middleware2.after_model()`
  10. `middleware1.after_model()`

  **智能体循环结束**
  11. `middleware3.after_agent()`
  12. `middleware2.after_agent()`
  13. `middleware1.after_agent()`
</Accordion>

**核心规则：**
* `before_*` 系列钩子函数：按中间件列表顺序执行（从第一个到最后一个）
* `after_*` 系列钩子函数：按中间件列表逆序执行（从最后一个到第一个）
* `wrap_*` 系列钩子函数：按嵌套方式执行（第一个中间件包裹其他所有中间件）

## 智能体跳转
如需在中间件中提前退出执行流程，可返回一个包含`jump_to`字段的字典：

**可用跳转目标：**
* `'end'`：跳转到智能体执行流程的末尾（或第一个`after_agent`钩子函数）
* `'tools'`：跳转到工具节点
* `'model'`：跳转到模型节点（或第一个`before_model`钩子函数）

<Tabs>
  <Tab title="装饰器写法">
    ```python  theme={null}
    from langchain.agents.middleware import after_model, hook_config, AgentState
    from langchain.messages import AIMessage
    from langgraph.runtime import Runtime
    from typing import Any


    @after_model
    @hook_config(can_jump_to=["end"])
    def check_for_blocked(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
        last_message = state["messages"][-1]
        if "BLOCKED" in last_message.content:
            return {
                "messages": [AIMessage("我无法响应该请求。")],
                "jump_to": "end"
            }
        return None
    ```
  </Tab>

  <Tab title="类写法">
    ```python  theme={null}
    from langchain.agents.middleware import AgentMiddleware, hook_config, AgentState
    from langchain.messages import AIMessage
    from langgraph.runtime import Runtime
    from typing import Any

    class BlockedContentMiddleware(AgentMiddleware):
        @hook_config(can_jump_to=["end"])
        def after_model(self, state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
            last_message = state["messages"][-1]
            if "BLOCKED" in last_message.content:
                return {
                    "messages": [AIMessage("我无法响应该请求。")],
                    "jump_to": "end"
                }
            return None
    ```
  </Tab>
</Tabs>

## 最佳实践
1.  保持中间件功能单一性——每个中间件专注实现一个功能
2.  优雅处理异常——避免因中间件异常导致智能体崩溃
3.  **选择合适的钩子函数类型**
    * 节点式钩子函数：适用于日志记录、参数校验等顺序执行逻辑
    * 包装式钩子函数：适用于重试、降级、缓存等流程控制场景
4.  清晰标注所有自定义状态属性
5.  集成到智能体前，先对中间件进行独立单元测试
6.  考虑执行顺序——将核心中间件放在列表首位
7.  优先使用内置中间件

## 实战案例

### 动态模型选择

<Tabs>
  <Tab title="装饰器写法">
    ```python  theme={null}
    from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
    from langchain.chat_models import init_chat_model
    from typing import Callable


    complex_model = init_chat_model("gpt-4o")
    simple_model = init_chat_model("gpt-4o-mini")

    @wrap_model_call
    def dynamic_model(
        request: ModelRequest,
        handler: Callable[[ModelRequest], ModelResponse],
    ) -> ModelResponse:
        # 根据对话长度选择不同模型
        if len(request.messages) > 10:
            model = complex_model
        else:
            model = simple_model
        return handler(request.override(model=model))
    ```
  </Tab>

  <Tab title="类写法">
    ```python  theme={null}
    from langchain.agents.middleware import AgentMiddleware, ModelRequest, ModelResponse
    from langchain.chat_models import init_chat_model
    from typing import Callable

    complex_model = init_chat_model("gpt-4o")
    simple_model = init_chat_model("gpt-4o-mini")

    class DynamicModelMiddleware(AgentMiddleware):
        def wrap_model_call(
            self,
            request: ModelRequest,
            handler: Callable[[ModelRequest], ModelResponse],
        ) -> ModelResponse:
            # 根据对话长度选择不同模型
            if len(request.messages) > 10:
                model = complex_model
            else:
                model = simple_model
            return handler(request.override(model=model))
    ```
  </Tab>
</Tabs>

### 工具调用监控

<Tabs>
  <Tab title="装饰器写法">
    ```python  theme={null}
    from langchain.agents.middleware import wrap_tool_call
    from langchain.tools.tool_node import ToolCallRequest
    from langchain.messages import ToolMessage
    from langgraph.types import Command
    from typing import Callable


    @wrap_tool_call
    def monitor_tool(
        request: ToolCallRequest,
        handler: Callable[[ToolCallRequest], ToolMessage | Command],
    ) -> ToolMessage | Command:
        print(f"正在执行工具: {request.tool_call['name']}")
        print(f"工具参数: {request.tool_call['args']}")
        try:
            result = handler(request)
            print(f"工具执行成功")
            return result
        except Exception as e:
            print(f"工具执行失败: {e}")
            raise
    ```
  </Tab>

  <Tab title="类写法">
    ```python  theme={null}
    from langchain.tools.tool_node import ToolCallRequest
    from langchain.agents.middleware import AgentMiddleware
    from langchain.messages import ToolMessage
    from langgraph.types import Command
    from typing import Callable

    class ToolMonitoringMiddleware(AgentMiddleware):
        def wrap_tool_call(
            self,
            request: ToolCallRequest,
            handler: Callable[[ToolCallRequest], ToolMessage | Command],
        ) -> ToolMessage | Command:
            print(f"正在执行工具: {request.tool_call['name']}")
            print(f"工具参数: {request.tool_call['args']}")
            try:
                result = handler(request)
                print(f"工具执行成功")
                return result
            except Exception as e:
                print(f"工具执行失败: {e}")
                raise
    ```
  </Tab>
</Tabs>

### 动态工具选择
在运行时选择相关工具，提升智能体执行性能和准确性。

**优势：**
* **精简提示词**：仅暴露相关工具，降低提示词复杂度
* **提高准确性**：模型从更少的工具选项中选择，决策更精准
* **权限控制**：基于用户权限动态过滤工具

<Tabs>
  <Tab title="装饰器写法">
    ```python  theme={null}
    from langchain.agents import create_agent
    from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
    from typing import Callable


    @wrap_model_call
    def select_tools(
        request: ModelRequest,
        handler: Callable[[ModelRequest], ModelResponse],
    ) -> ModelResponse:
        """基于状态/上下文选择相关工具的中间件"""
        # 根据状态和上下文选择少量相关工具
        relevant_tools = select_relevant_tools(request.state, request.runtime)
        return handler(request.override(tools=relevant_tools))

    agent = create_agent(
        model="gpt-4o",
        tools=all_tools,  # 需提前注册所有可用工具
        middleware=[select_tools],
    )
    ```
  </Tab>

  <Tab title="类写法">
    ```python  theme={null}
    from langchain.agents import create_agent
    from langchain.agents.middleware import AgentMiddleware, ModelRequest, ModelResponse
    from typing import Callable


    class ToolSelectorMiddleware(AgentMiddleware):
        def wrap_model_call(
            self,
            request: ModelRequest,
            handler: Callable[[ModelRequest], ModelResponse],
        ) -> ModelResponse:
            """基于状态/上下文选择相关工具的中间件"""
            # 根据状态和上下文选择少量相关工具
            relevant_tools = select_relevant_tools(request.state, request.runtime)
            return handler(request.override(tools=relevant_tools))

    agent = create_agent(
        model="gpt-4o",
        tools=all_tools,  # 需提前注册所有可用工具
        middleware=[ToolSelectorMiddleware()],
    )
    ```
  </Tab>
</Tabs>

### 系统消息处理
可通过`ModelRequest`对象的`system_message`字段在中间件中修改系统消息。无论智能体初始化时传入的是字符串类型的`system_prompt`，`system_message`字段始终存储的是一个 [`SystemMessage`](https://reference.langchain.com/python/langchain/messages/#langchain.messages.SystemMessage) 对象。

**案例：为系统消息添加上下文信息**

<Tabs>
  <Tab title="装饰器写法">
    ```python  theme={null}
    from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
    from langchain.messages import SystemMessage
    from typing import Callable


    @wrap_model_call
    def add_context(
        request: ModelRequest,
        handler: Callable[[ModelRequest], ModelResponse],
    ) -> ModelResponse:
        # 操作内容块列表
        new_content = list(request.system_message.content_blocks) + [
            {"type": "text", "text": "补充上下文信息。"}
        ]
        new_system_message = SystemMessage(content=new_content)
        return handler(request.override(system_message=new_system_message))
    ```
  </Tab>

  <Tab title="类写法">
    ```python  theme={null}
    from langchain.agents.middleware import AgentMiddleware, ModelRequest, ModelResponse
    from langchain.messages import SystemMessage
    from typing import Callable


    class ContextMiddleware(AgentMiddleware):
        def wrap_model_call(
            self,
            request: ModelRequest,
            handler: Callable[[ModelRequest], ModelResponse],
        ) -> ModelResponse:
            # 操作内容块列表
            new_content = list(request.system_message.content_blocks) + [
                {"type": "text", "text": "补充上下文信息。"}
            ]
            new_system_message = SystemMessage(content=new_content)
            return handler(request.override(system_message=new_system_message))
    ```
  </Tab>
</Tabs>

**案例：缓存控制（适用于Anthropic模型）**
使用Anthropic模型时，可通过带缓存控制指令的结构化内容块，对大型系统提示词进行缓存：

<Tabs>
  <Tab title="装饰器写法">
    ```python  theme={null}
    from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
    from langchain.messages import SystemMessage
    from typing import Callable


    @wrap_model_call
    def add_cached_context(
        request: ModelRequest,
        handler: Callable[[ModelRequest], ModelResponse],
    ) -> ModelResponse:
        # 操作内容块列表
        new_content = list(request.system_message.content_blocks) + [
            {
                "type": "text",
                "text": "以下是待分析的大型文档内容:\n\n<document>...</document>",
                # 此前的内容将被缓存
                "cache_control": {"type": "ephemeral"}
            }
        ]

        new_system_message = SystemMessage(content=new_content)
        return handler(request.override(system_message=new_system_message))
    ```
  </Tab>

  <Tab title="类写法">
    ```python  theme={null}
    from langchain.agents.middleware import AgentMiddleware, ModelRequest, ModelResponse
    from langchain.messages import SystemMessage
    from typing import Callable


    class CachedContextMiddleware(AgentMiddleware):
        def wrap_model_call(
            self,
            request: ModelRequest,
            handler: Callable[[ModelRequest], ModelResponse],
        ) -> ModelResponse:
            # 操作内容块列表
            new_content = list(request.system_message.content_blocks) + [
                {
                    "type": "text",
                    "text": "以下是待分析的大型文档内容:\n\n<document>...</document>",
                    "cache_control": {"type": "ephemeral"}  # 该部分内容将被缓存
                }
            ]

            new_system_message = SystemMessage(content=new_content)
            return handler(request.override(system_message=new_system_message))
    ```
  </Tab>
</Tabs>

**注意事项：**
* 无论智能体初始化时传入的是字符串类型的`system_prompt`，`ModelRequest.system_message`始终是一个 [`SystemMessage`](https://reference.langchain.com/python/langchain/messages/#langchain.messages.SystemMessage) 对象
* 可通过`SystemMessage.content_blocks`属性，以内容块列表的形式访问系统消息内容，无论原始内容是字符串还是列表格式
* 修改系统消息时，建议使用`content_blocks`属性并追加新的内容块，以保留原有内容结构
* 对于缓存控制等高级场景，可直接将 [`SystemMessage`](https://reference.langchain.com/python/langchain/messages/#langchain.messages.SystemMessage) 对象传入`create_agent`函数的`system_prompt`参数

## 拓展资源
* [中间件API参考文档](https://reference.langchain.com/python/langchain/middleware/)
* [内置中间件列表](/oss/python/langchain/middleware/built-in)
* [智能体测试指南](/oss/python/langchain/test)

***

<Callout icon="pen-to-square" iconType="regular">
  [在GitHub上编辑本页内容](https://github.com/langchain-ai/docs/edit/main/src/oss/langchain/middleware/custom.mdx)
</Callout>

<Tip icon="terminal" iconType="regular">
  [通过MCP将这些文档接入Claude、VSCode等工具](/use-these-docs)，获取实时解答
</Tip>


---

> 如需查看本文档的导航目录及其他页面，可获取llms.txt文件：https://docs.langchain.com/llms.txt